### Movie Reviews

In [ ]:
import nltk
from nltk.corpus import movie_reviews
import random

In [ ]:
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]


In [ ]:
review_sample = documents[0][0]

#### Tokenize

In [ ]:
from nltk.tag import pos_tag

print(pos_tag(review_sample))

#### Normalize

In [ ]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(review_sample))

#### Remove noise

In [ ]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(review_sample, stop_words))

In [143]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

neg_cleaned_tokens_list = [remove_noise(document[0], stop_words) for document in documents if document[1] == 'neg']
pos_cleaned_tokens_list = [remove_noise(document[0], stop_words) for document in documents if document[1] == 'pos']


#### Determining word density

In [146]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

pos_words = get_all_words(pos_cleaned_tokens_list)

In [147]:
from nltk import FreqDist

freq_pos = FreqDist(pos_words)
print(freq_pos.most_common(10))

[('film', 6171), ('movie', 3160), ('one', 3155), ('make', 2166), ('character', 2064), ('like', 1940), ('see', 1790), ('get', 1752), ('good', 1642), ('time', 1571)]


#### Training model

In [175]:
def get_words_for_model(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tokens)

pos_tokens_for_model = get_tweets_for_model(pos_cleaned_tokens_list)
neg_tokens_for_model = get_tweets_for_model(neg_cleaned_tokens_list)

In [176]:
import random

pos_dataset = [(doc_dict, "Positive")
                     for doc_dict in pos_tokens_for_model]

neg_dataset = [(doc_dict, "Negative")
                     for doc_dict in neg_tokens_for_model]

dataset = pos_dataset + neg_dataset

random.shuffle(dataset)
ntrain = (3 * len(dataset)) // 4

train_data = dataset[:ntrain]
test_data = dataset[ntrain:]

In [190]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.712
Most Informative Features
             outstanding = True           Positi : Negati =     14.4 : 1.0
            manipulation = True           Positi : Negati =     12.8 : 1.0
               ludicrous = True           Negati : Positi =     11.7 : 1.0
            breathtaking = True           Positi : Negati =     10.6 : 1.0
             uninvolving = True           Negati : Positi =     10.6 : 1.0
                   stink = True           Negati : Positi =      9.9 : 1.0
                   mulan = True           Positi : Negati =      9.4 : 1.0
                  spacey = True           Positi : Negati =      9.4 : 1.0
               uplifting = True           Positi : Negati =      9.4 : 1.0
                  alicia = True           Negati : Positi =      9.3 : 1.0
None


In [193]:
from nltk.tokenize import word_tokenize

# Get the first review id
fileids = [fileid for fileid in movie_reviews.fileids('neg')]
f = open(movie_reviews.abspath(fileids[0]), "r")
review0 = f.read()

fileids = [fileid for fileid in movie_reviews.fileids('pos')]
f = open(movie_reviews.abspath(fileids[0]), "r")
review1 = f.read()

# Check if classifier this review correctly
custom_tokens = remove_noise(word_tokenize(review0))

print(classifier.classify(dict([token, True] for token in custom_tokens)))
print(review0)

# Check if classifier this review correctly
custom_tokens = remove_noise(word_tokenize(review1))

print('-------------')
print(classifier.classify(dict([token, True] for token in custom_tokens)))
print(review1)

Negative
plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . 
they seem to have taken this pretty neat concept , but executed it terribly . 
so what are the problems with the movie ? 
well , its main problem is that it's simply too jumbled . 
it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , h

### Twitter

In [ ]:
from nltk.corpus import twitter_samples

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

- negative_tweets.json: 5000 tweets with negative sentiments
- positive_tweets.json: 5000 tweets with positive sentiments
- tweets.20150430-223406.json: 20000 tweets with no sentiments

In [ ]:
import nltk

#### Tokenize

In [ ]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [ ]:
from nltk.tag import pos_tag

print(pos_tag(tweet_tokens[0]))

From the list of tags, here is the list of the most common items and their meaning:

- NNP: Noun, proper, singular
- NN: Noun, common, singular or mass
- IN: Preposition or conjunction, subordinating
- VBG: Verb, gerund or present participle
- VBN: Verb, past participle

#### Normalize

In [ ]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

#### Remove noise

In [ ]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

#### Determining word density

In [ ]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [ ]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

#### Training model

In [ ]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [ ]:
import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [ ]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

In [ ]:
from nltk.tokenize import word_tokenize

custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

### Reference

https://www.datacamp.com/community/tutorials/simplifying-sentiment-analysis-python

https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk
